In [1]:
from os import path


import logging

import pandas as pd

In [2]:
import sys
sys.path.append("../") # go to parent dir

In [3]:
from dependencynet.schema import SchemaBuilder
from dependencynet.model import ModelBuilder, ModelStorageService
from dependencynet.trees_utils import build_tree, pretty_print_tree

In [4]:
logging.basicConfig()
logger = logging.getLogger('dependencynet')
logger.setLevel(logging.INFO)

In [5]:
filename = path.join('..', 'tests', 'resources', 'data', 'compact', 'towns.csv')
data = pd.read_csv(filename, delimiter=';')

properties = ['area', 'country', 'town']
source_df = pd.DataFrame(data, columns=properties)

display(source_df)

,area,country,town
0,Europe,France,Paris
1,Europe,France,Lyon
2,Europe,UK,London
3,Europe,Italia,Roma
4,Asia,Japan,Tokyo


In [6]:
schema_towns = SchemaBuilder().level('area', 'A').level('country', 'C').level('town', 'T').resource('monument', 'M').render()

model = ModelBuilder().from_compact(source_df) \
                      .with_schema(schema_towns) \
                      .render()

dfs = model.levels_datasets

if logger.level <= logging.INFO:
    logger.info('main resultats')
    display(dfs[0])
    display(dfs[1])
    display(dfs[2])

INFO:dependencynet.schema:rendering schema
INFO:dependencynet.model:extract_items_root keys=['area'] id_pattern=A{id:02d} => shape=(2, 4)
INFO:dependencynet.model:extract_items_root keys=['area', 'country'] id_pattern={id_parent}C{id:02d} => shape=(4, 6)
INFO:dependencynet.model:extract_items_root keys=['area', 'country', 'town'] id_pattern={id_parent}T{id:02d} => shape=(5, 7)
INFO:dependencynet:main resultats


,area,pos,id,label
0,Europe,1,A01,A01 Europe
4,Asia,2,A02,A02 Asia


,area,country,pos,id_parent,id,label
0,Europe,France,1,A01,A01C01,A01C01 France
1,Europe,UK,2,A01,A01C02,A01C02 UK
2,Europe,Italia,3,A01,A01C03,A01C03 Italia
3,Asia,Japan,1,A02,A02C01,A02C01 Japan


,area,country,town,pos,id_parent,id,label
0,Europe,France,Paris,1,A01C01,A01C01T01,A01C01T01 Paris
1,Europe,France,Lyon,2,A01C01,A01C01T02,A01C01T02 Lyon
2,Europe,UK,London,1,A01C02,A01C02T01,A01C02T01 London
3,Europe,Italia,Roma,1,A01C03,A01C03T01,A01C03T01 Roma
4,Asia,Japan,Tokyo,1,A02C01,A02C01T01,A02C01T01 Tokyo


In [7]:
folder_name = path.join('output', 'datasets')
storage = ModelStorageService(folder_name)
storage.save(model)

INFO:dependencynet.model:model folder is output/datasets/current
INFO:dependencynet.model:dateset saved under name output/datasets/current/area.csv
INFO:dependencynet.model:dateset saved under name output/datasets/current/country.csv
INFO:dependencynet.model:dateset saved under name output/datasets/current/town.csv


In [8]:
tree = build_tree(dfs, keys)
display(tree)

NameError: name 'keys' is not defined

In [ ]:
pretty_print_tree(tree, keys)